<a href="https://colab.research.google.com/github/radanim/TIL/blob/master/Conv1D%2BLSTM_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Conv1D, Flatten
import os
import glob

In [3]:
output=glob.glob('/content/drive/MyDrive/dataset/train 01/*')

# 데이터 프레임 통합하기
df_total = pd.read_csv(output[0])
for i in range (1,100):
    df=pd.read_csv(output[i])
    type(df)
    df_total = pd.concat([df_total, df], ignore_index=True, axis=0)

df_total.shape

# 학습데이터 x,y 분리 
dfx = df_total.drop(columns="FREL(2)")
dfy = df_total["FREL(2)"]
print(dfx.shape, dfy.shape) # (1076, 14), (1076, )

# MinMaxScaler
from gc import callbacks
from sklearn.preprocessing import MinMaxScaler
# 복원을 위해 x, y 따로 스케일링하기
# 스케일링 후 list 형태로 변환하기
mms = MinMaxScaler()
x = mms.fit_transform(dfx).tolist()
y = mms.fit_transform(dfy.to_frame()).tolist() # Series > frame > list, Series 상태에서는 minmaxscaler 적용안됨.

print(len(x), len(y)) # 1076, 1076

window_size = 150
len(y) - window_size

# window_size 만큼만 슬라이싱
# 10step x값으로 다음 step y값을 예측
# 다음 날 종가(i+windows_size)는 포함되지 않도록 X값을 만들고
# 다음 날 종가를 y로 만듦
# window_size = 10 

x_data = []
y_data = []
for i in range(len(y) - window_size):
    # print(i, end=",")
    _x = x[i : i+window_size]
    _y = y[i+window_size]
    x_data.append(_x)
    y_data.append(_y)


(106393, 14) (106393,)
106393 106393


In [4]:
# 경로 설정

a=df_total[df_total.TIME == 0.0].index[76]
print(a)

b=df_total[df_total.TIME == 0.0].index[96]
print(b)

# train, test, valid
X_train = np.array(x_data[:a]) # 80
y_train = np.array(y_data[:a]) #80
X_valid = np.array(x_data[a:b]) # 80
y_valid = np.array(y_data[a:b]) #80
X_test = np.array(x_data[b:]) # 20
y_test = np.array(y_data[b:]) #20
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape

# 성능지표 함수
def s_mape(y_true, y_pred):
  ja = tf.math.abs(y_pred - y_true)
  mo = (tf.math.abs(y_true)+tf.math.abs(y_pred))/2
  loss = 100* tf.reduce_mean(ja/mo)
  return loss


81205
102044


In [5]:
data_size = dfx.shape[1]
print(data_size)

14


In [6]:
# 모델 생성 (validation 데이터 범위 지정해서 돌리기)

data_size = dfx.shape[1]

model = Sequential()
model.add(LSTM(units=32, activation='relu', return_sequences=True, input_shape=(window_size, data_size))) # bi 히든레이어 unit 10+10=20 
model.add(Dropout(0.3))

model.add(LSTM(units=32, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(Dense(units=32))

model.add(Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation="relu")) #유닛수=히든레이어수
model.add(Dropout(0.3))
model.add(Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation="relu")) #유닛수=히든레이어수
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(units=1)) # 분류를 하기 위한 학습, output layer = FNN Fully Connected Neural Network
model.summary()

model.compile(optimizer = 'adam',
              loss = 'mse',
              metrics = [s_mape]) # 회귀 (y^ - y)^2 // 분류 categorical entropy
model.save('model100.h5')



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 150, 32)           6016      
                                                                 
 dropout (Dropout)           (None, 150, 32)           0         
                                                                 
 lstm_1 (LSTM)               (None, 150, 32)           8320      
                                                                 
 dropout_1 (Dropout)         (None, 150, 32)           0         
                                                                 
 dense (Dense)               (None, 150, 32)           1056      
                                                                 
 conv1d (Conv1D)             (None, 150, 64)           6208      
                                                                 
 dropout_2 (Dropout)         (None, 150, 64)           0

In [ ]:
print(X_train.shape, y_train.shape)
file_path = '/content/drive/MyDrive/dataset/train 01'
# 모델 epoch별 가중치 저장 체크포인트
cp_callback = [tf.keras.callbacks.ModelCheckpoint(file_path ,save_weights_only=True,verbose=1),
tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)]

#early stopping patience loss값이 연속적으로 patience 횟수만큼 커질때 스탑 
history = model.fit(X_train, y_train, validation_data= (X_valid, y_valid), epochs = 500, batch_size=30, verbose=1, callbacks=cp_callback) 
#verbose progress 바가 생기는게 1 아닌게 0 결과만 2 

(81205, 150, 14) (81205, 1)
Epoch 1/500
2707/2707 [==============================] - ETA: 0s - loss: 5.3108e-04 - s_mape: 70.7769
Epoch 1: saving model to /content/drive/MyDrive/dataset/train 01
2707/2707 [==============================] - 1307s 483ms/step - loss: 5.3108e-04 - s_mape: 70.7769 - val_loss: 0.0034 - val_s_mape: 59.1205
Epoch 2/500
2707/2707 [==============================] - ETA: 0s - loss: 1.0451e-04 - s_mape: 57.0655
Epoch 2: saving model to /content/drive/MyDrive/dataset/train 01
2707/2707 [==============================] - 1268s 469ms/step - loss: 1.0451e-04 - s_mape: 57.0655 - val_loss: 0.0035 - val_s_mape: 58.8187
Epoch 3/500
2707/2707 [==============================] - ETA: 0s - loss: 6.2191e-05 - s_mape: 57.2415
Epoch 3: saving model to /content/drive/MyDrive/dataset/train 01
2707/2707 [==============================] - 1235s 456ms/step - loss: 6.2191e-05 - s_mape: 57.2415 - val_loss: 0.0038 - val_s_mape: 58.4961
Epoch 4/500
2707/2707 [============================

In [ ]:
history_df=pd.DataFrame(history.history)
history_df

loss_final=history.history['loss'][-1]
val_loss_final =history.history['val_loss'][-1]
results=model.evaluate(X_test,y_test,batch_size=128)
print(results)

In [ ]:
# 모델 설계 및 학습 코드와 시각화 코드 분리 
time_index = df_total[b:-window_size]['TIME']
time_index.shape

# y_pred
# df_hist = pd.DataFrame(history.history)
# df_hist['loss'].plot()
y_pred = model.predict(X_test)

plt.figure(figsize= (12, 4))
plt.title("Loss/Val_loss")
plt.xlabel("epochs")
plt.ylabel("loss/val_loss")
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.text(17, 0.01, 'loss={:.4f} val_loss={:.2f}'.format(val_loss_final, val_loss_final), fontsize=14)
plt.legend()
plt.savefig('./valid_loss_epoch500_6.png', dpi=500)

# rmse
# np.sqrt(np.square(y_test - y_pred).mean())
# print(y_test.shape, y_pred.shape) # (214, 1), (214, 1)

plt.figure(figsize= (12, 4))
plt.title("CS/I Release Prediction")
plt.xlabel("time")
plt.ylabel("CS/I FREL")
plt.plot(time_index,y_pred.flatten(), label="predicted CS/I Release")
plt.plot(time_index,y_test.flatten(), label="real CS/I Release")
plt.text(200000, 0.200, 'loss={:.4f} s_mape={:.2f}'.format(results[0], results[1]), fontsize=14)
plt.legend()
plt.savefig('./scaling_result_epoch500_6.png', dpi=500)

y_predict_inverse = mms.inverse_transform(y_pred)
y_predict_inverse[:5]

y_test_inverse = mms.inverse_transform(y_test)
y_test_inverse[:5]

# rmse 
# np.sqrt(np.square(y_test_inverse - y_predict_inverse).mean())

# 시각화
plt.figure(figsize = (12, 3))
plt.title("CS/I Release Prediction")
plt.xlabel("time")
plt.ylabel("CS/I FREL")
plt.plot(time_index,y_predict_inverse, label = "predicted CS/I Release")
plt.plot(time_index,y_test_inverse, label="real CS/I Release")
plt.text(200000, .0000200, 'loss={:.4f} s_mape={:.2f}'.format(results[0], results[1]), fontsize=14)
plt.legend()
plt.savefig('./inverse_result_epoch500_6.png', dpi=500)